In [ ]:
# Install ultralytics and torch libraries
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install git+https://github.com/ultralytics/ultralytics.git@main
!pip install easyocr

# Import necessary libraries
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from ultralytics import YOLO
import io
import os
import requests
import easyocr
from PIL import Image
import logging
from concurrent.futures import ThreadPoolExecutor

# Check if CUDA is available and set device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

# Load a pre-trained YOLO model (placeholder for now)
#model = YOLO("yolo11n.pt")

# Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

# Load your custom model
model = YOLO("/content/drive/MyDrive/Detection_License_Plate_model/best.pt")

Looking in indexes: https://download.pytorch.org/whl/cu121
  Cloning https://github.com/ultralytics/ultralytics.git (to revision main) to /tmp/pip-req-build-czhmlzn8
  Running command git clone --filter=blob:none --quiet https://github.com/ultralytics/ultralytics.git /tmp/pip-req-build-czhmlzn8
  Resolved https://github.com/ultralytics/ultralytics.git to commit 04d7fcb7af34635ac9a4409eb0e975cbbf24f38a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Using device: cpu
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# With image enhancements // OCR gpu is nulled and super resolution is nulled

!wget https://github.com/Saafke/EDSR_Tensorflow/raw/master/models/EDSR_x4.pb
from cv2.dnn_superres import DnnSuperResImpl_create

# Initialize EasyOCR reader with GPU enabled for faster processing
reader = easyocr.Reader(['en'])#, gpu=True)

# Set up logging
logging.basicConfig(filename='processing.log', level=logging.INFO,
                    format='%(asctime)s - %(message)s')

# Initialize Super-Resolution model
sr = DnnSuperResImpl_create()
sr.readModel("EDSR_x4.pb")  # Pre-trained model
sr.setModel("edsr", 4)  # 4x upscaling

# Function to enhance image contrast using CLAHE
def enhance_contrast(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    l = clahe.apply(l)
    lab = cv2.merge((l, a, b))
    return cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)

# Function for gamma correction
def gamma_correction(image, gamma=1.5):
    inv_gamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** inv_gamma) * 255 for i in range(256)]).astype("uint8")
    return cv2.LUT(image, table)

# Function to preprocess an image
def preprocess_plate(plate_region):
    plate_region = enhance_contrast(plate_region)
    plate_region = gamma_correction(plate_region, gamma=1.2)
    gray = cv2.cvtColor(plate_region, cv2.COLOR_BGR2GRAY)
    preprocessed_image = cv2.adaptiveThreshold(
        gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2
    )
    return preprocessed_image

# Function to enhance plate region with super-resolution
def super_resolve(plate_region):
    return sr.upsample(plate_region)

# Function to validate and format license plate text
def validate_and_format_text(text):
    allowed_characters = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789- "  # License plate-specific characters
    filtered_text = ''.join([char for char in text.upper() if char in allowed_characters])
    if len(filtered_text) >= 5:  # Validate based on region-specific patterns
        return filtered_text
    return None

# Function to process a single image
def process_image(image, model, show_preprocessing=False):
    results = model.predict(image, conf=0.15, iou=0.25)  # Fine-tuned thresholds
    detected_texts = []
    fixed_plate_resolution = (240, 120)  # Fixed resolution for plate images

    for r in results:
        if hasattr(r, 'boxes'):
            for i, box in enumerate(r.boxes):
                x1, y1, x2, y2 = map(int, box.xyxy[0])

                # Add dynamic padding
                padding = int(0.05 * min(x2 - x1, y2 - y1))
                x1, y1 = max(0, x1 - padding), max(0, y1 - padding)
                x2, y2 = min(image.shape[1], x2 + padding), min(image.shape[0], y2 + padding)

                # Preprocess the cropped region
                plate_region = image[y1:y2, x1:x2]
                #plate_region = super_resolve(plate_region)  # Apply super-resolution
                preprocessed_image = preprocess_plate(plate_region)

                # Perform OCR
                ocr_results = reader.readtext(preprocessed_image, detail=1)

                for _, text, _ in ocr_results:
                    formatted_text = validate_and_format_text(text)
                    if formatted_text:
                        detected_texts.append(formatted_text)

                        # Display cropped plate region above detection box
                        overlay_x1 = max(0, x1)
                        overlay_x2 = min(image.shape[1], x1 + fixed_plate_resolution[0])
                        overlay_y1 = max(0, y1 - fixed_plate_resolution[1] - 50)
                        overlay_y2 = y1 - 50

                        if overlay_y1 >= 0:  # Ensure the overlay doesn't go out of bounds
                            resized_plate = cv2.resize(plate_region, fixed_plate_resolution)
                            image[overlay_y1:overlay_y2, overlay_x1:overlay_x2] = resized_plate

                        # Draw bounding boxes and text above cropped plate
                        text_size = cv2.getTextSize(formatted_text, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)[0]
                        text_x, text_y = overlay_x1, overlay_y1 - 10
                        cv2.rectangle(
                            image,
                            (text_x, text_y - text_size[1] - 5),
                            (text_x + text_size[0] + 5, text_y + 5),
                            (0, 255, 0), -1
                        )
                        cv2.putText(
                            image, formatted_text, (text_x, text_y),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2
                        )

    return image, "\n".join(detected_texts)

# Function to save results
def save_results(file_path, processed_image, extracted_text, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    filename = os.path.basename(file_path)
    base_name = os.path.splitext(filename)[0]

    if processed_image is not None:
        processed_image_path = os.path.join(output_dir, f"{base_name}_processed.jpg")
        cv2.imwrite(processed_image_path, processed_image)

    text_file_path = os.path.join(output_dir, f"{base_name}_detected_plates.txt")
    with open(text_file_path, "w") as f:
        f.write(extracted_text)

    logging.info(f"Processed results saved for: {file_path}")

# Function to process images/videos from a folder or single file
def process_files(input_path, output_dir, model, show_preprocessing=False):
    if os.path.isfile(input_path):
        process_file(input_path, output_dir, model, show_preprocessing)
    elif os.path.isdir(input_path):
        files = [os.path.join(input_path, f) for f in os.listdir(input_path)
                 if f.lower().endswith((".jpg", ".jpeg", ".png", ".mp4"))]

        # Process files in parallel
        with ThreadPoolExecutor() as executor:
            for file_path in files:
                executor.submit(process_file, file_path, output_dir, model, show_preprocessing)
    else:
        logging.error(f"Invalid path: {input_path}")

# Function to process a single file (image or video)
def process_file(file_path, output_dir, model, show_preprocessing=False):
    if file_path.lower().endswith(('.jpg', '.jpeg', '.png')):
        image = cv2.imread(file_path)
        if image is not None:
            processed_image, extracted_text = process_image(image, model, show_preprocessing)
            save_results(file_path, processed_image, extracted_text, output_dir)
    elif file_path.lower().endswith('.mp4'):
        process_video(file_path, output_dir, model)

# Function to process video files
def process_video(video_path, output_dir, model):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    output_video_path = os.path.join(output_dir, f"{os.path.splitext(os.path.basename(video_path))[0]}_processed.mp4")
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Process every single frame
        processed_frame, _ = process_image(frame, model)
        out.write(processed_frame)

    cap.release()
    out.release()
    logging.info(f"Processed video saved to: {output_video_path}")

# Main execution
if __name__ == "__main__":
    from ultralytics import YOLO

    input_path = "/content/drive/MyDrive/Detection_License_Plate_model/number_plates_post_model_train"
    output_dir = "/content/drive/MyDrive/Detection_License_Plate_model/detected_results"


    # Process folder or single file
    process_files(input_path, output_dir, model, show_preprocessing=True)


--2024-12-14 11:17:33--  https://github.com/Saafke/EDSR_Tensorflow/raw/master/models/EDSR_x4.pb
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Saafke/EDSR_Tensorflow/master/models/EDSR_x4.pb [following]
--2024-12-14 11:17:33--  https://raw.githubusercontent.com/Saafke/EDSR_Tensorflow/master/models/EDSR_x4.pb
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38573255 (37M) [application/octet-stream]
Saving to: ‘EDSR_x4.pb.4’

EDSR_x4.pb.4        100%[===================>]  36.79M   144MB/s    in 0.3s    

2024-12-14 11:17:34 (144 MB/s) - ‘EDSR_x4.pb.4’ saved [38573255/38573255]








0: 448x640 1 License_Plate, 280.0ms
Speed: 7.3ms preprocess, 280.0ms inference, 1.4ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 License_Plate, 394.3ms
Speed: 8.8ms preprocess, 394.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 448)
0: 448x640 3 License_Plates, 399.0ms
Speed: 11.5ms preprocess, 399.0ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)

0: 640x448 1 License_Plate, 495.1ms
Speed: 24.4ms preprocess, 495.1ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 448)
0: 448x640 1 License_Plate, 494.5ms
Speed: 19.8ms preprocess, 494.5ms inference, 1.3ms postprocess per image at shape (1, 3, 448, 640)
0: 320x640 2 License_Plates, 659.4ms
Speed: 5.5ms preprocess, 659.4ms inference, 1.6ms postprocess per image at shape (1, 3, 320, 640)

0: 448x640 5 License_Plates, 620.2ms
Speed: 4.9ms preprocess, 620.2ms inference, 8.0ms postprocess per image at shape (1, 3, 448, 640)


0: 384x640 2 License_Plates, 530.3ms
Speed: 3.